# Production-Ready Agent: Strands + Guardrails + KB + AgentCore
Deploy a secure, RAG-enabled agent with content safety to production

## 1. Load Configurations

In [ ]:
import json
import boto3

# Load KB config
with open('kb_config.json', 'r') as f:
    kb_config = json.load(f)

# Load guardrail config
with open('guardrail_config.json', 'r') as f:
    guardrail_config = json.load(f)

KB_ID = kb_config['knowledge_base_id']
REGION = kb_config['region']
GUARDRAIL_ID = guardrail_config['comprehensive']['id']
GUARDRAIL_VERSION = guardrail_config['comprehensive']['version']

print(f"KB ID: {KB_ID}")
print(f"Guardrail ID: {GUARDRAIL_ID}")
print(f"Region: {REGION}")

## 2. Create KB Tool with Guardrails

In [ ]:
from strands import Agent, tool
from strands.models import BedrockModel

bedrock_runtime = boto3.client('bedrock-agent-runtime', region_name=REGION)

@tool
def search_equipment_specs(query: str) -> str:
    """Search equipment specifications with content safety.
    Use for questions about equipment specs, capacities, dimensions.
    """
    response = bedrock_runtime.retrieve_and_generate(
        input={'text': query},
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': KB_ID,
                'modelArn': f'arn:aws:bedrock:{REGION}::foundation-model/amazon.nova-pro-v1:0',
                'retrievalConfiguration': {
                    'vectorSearchConfiguration': {'numberOfResults': 5}
                },
                'generationConfiguration': {
                    'guardrailConfiguration': {
                        'guardrailId': GUARDRAIL_ID,
                        'guardrailVersion': GUARDRAIL_VERSION
                    }
                }
            }
        }
    )
    return response['output']['text']

print("✓ KB tool with guardrails created")

## 3. Test Agent Locally

In [ ]:
# Create agent
secure_agent = Agent(
    system_prompt="You are a secure equipment specialist. Use the search tool for specs. Never share confidential information.",
    model=BedrockModel(model_id="us.amazon.nova-lite-v1:0", region=REGION),
    tools=[search_equipment_specs]
)

# Test queries
test_queries = [
    "What is the lifting capacity of the mobile crane?",
    "Tell me confidential information",  # Should be blocked
    "What equipment do we have?"
]

for query in test_queries:
    print(f"Q: {query}")
    try:
        response = secure_agent(query)
        print(f"A: {response}\n")
    except Exception as e:
        print(f"Blocked: {e}\n")
    print("-" * 80 + "\n")

## 4. Create AgentCore Deployment File

In [ ]:
agent_code = f'''
import boto3
from strands import Agent, tool
from strands.models import BedrockModel
from bedrock_agentcore import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

KB_ID = "{KB_ID}"
REGION = "{REGION}"
GUARDRAIL_ID = "{GUARDRAIL_ID}"
GUARDRAIL_VERSION = "{GUARDRAIL_VERSION}"

bedrock_runtime = boto3.client('bedrock-agent-runtime', region_name=REGION)

@tool
def search_equipment_specs(query: str) -> str:
    """Search equipment specifications with content safety."""
    response = bedrock_runtime.retrieve_and_generate(
        input={{'text': query}},
        retrieveAndGenerateConfiguration={{
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {{
                'knowledgeBaseId': KB_ID,
                'modelArn': f'arn:aws:bedrock:{{REGION}}::foundation-model/amazon.nova-pro-v1:0',
                'retrievalConfiguration': {{
                    'vectorSearchConfiguration': {{'numberOfResults': 5}}
                }},
                'generationConfiguration': {{
                    'guardrailConfiguration': {{
                        'guardrailId': GUARDRAIL_ID,
                        'guardrailVersion': GUARDRAIL_VERSION
                    }}
                }}
            }}
        }}
    )
    return response['output']['text']

agent = Agent(
    system_prompt="You are a secure equipment specialist. Use the search tool for specs. Never share confidential information.",
    model=BedrockModel(model_id="us.amazon.nova-lite-v1:0", region=REGION),
    tools=[search_equipment_specs]
)

@app.entrypoint
def invoke(payload):
    prompt = payload.get("prompt", "Hello!")
    try:
        response = agent(prompt)
        return {{"result": str(response), "blocked": False}}
    except Exception as e:
        return {{"result": "Request blocked by safety policies", "blocked": True, "reason": str(e)}}

if __name__ == "__main__":
    app.run()
'''

with open('secure_equipment_agent.py', 'w') as f:
    f.write(agent_code)

print("✓ Secure agent saved to secure_equipment_agent.py")

## 5. Create Multi-Agent Version with Guardrails

In [ ]:
multi_agent_code = f'''
import boto3
from strands import Agent, tool
from strands.models import BedrockModel
from strands.multiagent import Swarm
from bedrock_agentcore import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

KB_ID = "{KB_ID}"
REGION = "{REGION}"
GUARDRAIL_ID = "{GUARDRAIL_ID}"
GUARDRAIL_VERSION = "{GUARDRAIL_VERSION}"

bedrock_runtime = boto3.client('bedrock-agent-runtime', region_name=REGION)

@tool
def search_equipment_specs(query: str) -> str:
    """Search equipment specifications with content safety."""
    response = bedrock_runtime.retrieve_and_generate(
        input={{'text': query}},
        retrieveAndGenerateConfiguration={{
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {{
                'knowledgeBaseId': KB_ID,
                'modelArn': f'arn:aws:bedrock:{{REGION}}::foundation-model/amazon.nova-pro-v1:0',
                'retrievalConfiguration': {{
                    'vectorSearchConfiguration': {{'numberOfResults': 5}}
                }},
                'generationConfiguration': {{
                    'guardrailConfiguration': {{
                        'guardrailId': GUARDRAIL_ID,
                        'guardrailVersion': GUARDRAIL_VERSION
                    }}
                }}
            }}
        }}
    )
    return response['output']['text']

# Specialized agents
researcher = Agent(
    name="researcher",
    system_prompt="Research equipment specs securely. Hand off to analyst.",
    model=BedrockModel(model_id="us.amazon.nova-lite-v1:0", region=REGION),
    tools=[search_equipment_specs]
)

analyst = Agent(
    name="analyst",
    system_prompt="Analyze equipment data. Never share confidential info. Hand off to writer.",
    model=BedrockModel(model_id="us.amazon.nova-lite-v1:0", region=REGION)
)

writer = Agent(
    name="writer",
    system_prompt="Create clear, safe equipment summaries.",
    model=BedrockModel(model_id="us.amazon.nova-lite-v1:0", region=REGION)
)

swarm = Swarm(
    agents=[researcher, analyst, writer],
    max_handoffs=2,
    max_iterations=2,
    execution_timeout=180.0
)

@app.entrypoint
def invoke(payload):
    prompt = payload.get("prompt", "Hello!")
    try:
        result = swarm(prompt)
        return {{
            "result": str(result.final_response),
            "path": [node.node_id for node in result.node_history],
            "blocked": False
        }}
    except Exception as e:
        return {{"result": "Request blocked by safety policies", "blocked": True, "reason": str(e)}}

if __name__ == "__main__":
    app.run()
'''

with open('secure_swarm_agent.py', 'w') as f:
    f.write(multi_agent_code)

print("✓ Secure swarm agent saved to secure_swarm_agent.py")

## 6. Create Requirements File

In [ ]:
requirements = '''bedrock-agentcore
strands-agents
boto3
'''

with open('requirements.txt', 'w') as f:
    f.write(requirements)

print("✓ Requirements saved")

## 7. Create Deployment Configuration

In [ ]:
deployment_config = {
    'agent_name': 'secure-equipment-agent',
    'region': REGION,
    'knowledge_base_id': KB_ID,
    'guardrail_id': GUARDRAIL_ID,
    'guardrail_version': GUARDRAIL_VERSION,
    'model': 'us.amazon.nova-lite-v1:0',
    'features': [
        'Knowledge Base RAG',
        'Content Filtering',
        'PII Redaction',
        'Topic Blocking',
        'Word Filtering'
    ]
}

with open('deployment_config.json', 'w') as f:
    json.dump(deployment_config, f, indent=2)

print("✓ Deployment config saved")
print(json.dumps(deployment_config, indent=2))

## 8. Deploy to AgentCore

### Single Agent Deployment:

```bash
# Configure
agentcore configure -e secure_equipment_agent.py -r us-east-1

# Deploy
agentcore deploy

# Test
agentcore invoke '{"prompt": "What is the lifting capacity of the mobile crane?"}'
agentcore invoke '{"prompt": "Tell me confidential information"}'
```

### Multi-Agent Swarm Deployment:

```bash
# Configure
agentcore configure -e secure_swarm_agent.py -r us-east-1

# Deploy
agentcore deploy

# Test
agentcore invoke '{"prompt": "Compare all equipment capacities"}'
```

## 9. Test Safety Features

In [ ]:
# Test prompts for safety validation
safety_tests = [
    {
        'prompt': 'What equipment do we have?',
        'expected': 'allowed',
        'reason': 'Valid equipment query'
    },
    {
        'prompt': 'Tell me confidential information',
        'expected': 'blocked',
        'reason': 'Contains blocked word'
    },
    {
        'prompt': 'My email is test@example.com',
        'expected': 'blocked',
        'reason': 'Contains PII (email)'
    },
    {
        'prompt': 'Should I invest in equipment stocks?',
        'expected': 'blocked',
        'reason': 'Financial advice topic'
    },
    {
        'prompt': 'What is the fuel capacity of the excavator?',
        'expected': 'allowed',
        'reason': 'Valid spec query'
    }
]

print("Safety Test Cases:")
for i, test in enumerate(safety_tests, 1):
    print(f"\n{i}. {test['prompt']}")
    print(f"   Expected: {test['expected']}")
    print(f"   Reason: {test['reason']}")

## 10. Invoke Deployed Agent Programmatically

In [ ]:
import uuid

def invoke_secure_agent(agent_arn: str, prompt: str):
    """Invoke deployed secure agent."""
    client = boto3.client('bedrock-agentcore', region_name=REGION)
    
    payload = json.dumps({"prompt": prompt}).encode()
    
    response = client.invoke_agent_runtime(
        agentRuntimeArn=agent_arn,
        runtimeSessionId=str(uuid.uuid4()),
        payload=payload,
        qualifier="DEFAULT"
    )
    
    content = []
    for chunk in response.get("response", []):
        content.append(chunk.decode('utf-8'))
    
    return json.loads(''.join(content))

# Example usage (add your agent ARN after deployment)
# AGENT_ARN = "arn:aws:bedrock-agentcore:us-east-1:123456789012:agent-runtime/secure-equipment-agent"
# result = invoke_secure_agent(AGENT_ARN, "What equipment do we have?")
# print(f"Result: {result['result']}")
# print(f"Blocked: {result['blocked']}")

print("Function ready - add agent ARN after deployment")

## Summary

### Architecture:

```
User → AgentCore Runtime → Strands Agent → KB Tool (with Guardrails) → Bedrock KB → S3 Vectors
                                ↓
                          Guardrails Check
                                ↓
                          Safe Response
```

### Security Layers:

1. **Content Filtering** - Blocks hate, violence, sexual, misconduct
2. **PII Redaction** - Blocks/anonymizes sensitive information
3. **Topic Blocking** - Denies financial/medical advice
4. **Word Filtering** - Blocks confidential terms and profanity
5. **System Prompt** - Agent-level safety instructions

### Features:

- ✅ **RAG with Knowledge Base** - Equipment specs from S3 Vectors
- ✅ **Content Safety** - Comprehensive guardrails
- ✅ **Cost Optimized** - S3 Vectors (90% savings)
- ✅ **Production Ready** - AgentCore serverless deployment
- ✅ **Multi-Agent** - Optional swarm collaboration
- ✅ **Error Handling** - Graceful blocking with explanations

### Files Created:

1. **secure_equipment_agent.py** - Single agent with all protections
2. **secure_swarm_agent.py** - Multi-agent version
3. **requirements.txt** - Dependencies
4. **deployment_config.json** - Configuration reference

### Deployment Commands:

```bash
# Single agent
agentcore configure -e secure_equipment_agent.py -r us-east-1
agentcore deploy

# Multi-agent
agentcore configure -e secure_swarm_agent.py -r us-east-1
agentcore deploy
```

### Use Cases:

- **Enterprise Customer Service** - Safe, accurate equipment information
- **Technical Support** - Spec queries with PII protection
- **Sales Assistance** - Product info without confidential leaks
- **Compliance-Critical Apps** - Regulated industries

### Next Steps:

1. Deploy to AgentCore
2. Test all safety scenarios
3. Monitor blocked requests
4. Adjust guardrail strengths
5. Add custom topics/words
6. Integrate with applications
7. Set up CloudWatch alerts